In [1]:
import gym
from agent import Agent
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

2023-11-13 16:27:49.333546: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def describe_env(env: gym.Env):
    num_actions = env.action_space.n
    obs = env.observation_space
    num_obs = env.observation_space.shape

    print("Observation space: ", obs)
    print("Observation space size: ", num_obs)
    print("Number of actions: ", num_actions)

    return num_obs, num_actions

In [3]:
def plot(results, result_index, window_size, title):
  plt.figure(figsize=(25, 10))

  for result in results:
      sns.lineplot(np.convolve(result[result_index], np.ones(window_size) / window_size, mode='same'), label=f"(LR, EF) {(result[2]) }")
      
  plt.title(title)
  plt.legend()
  plt.show()

In [4]:

env = gym.make('MsPacman-v4')

num_obs, num_actions = describe_env(env)

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


Observation space:  Box(0, 255, (210, 160, 3), uint8)
Observation space size:  (210, 160, 3)
Number of actions:  9


In [7]:

agent = Agent({
  "num_obs": num_obs,
  "num_actions": num_actions,
  "learning_rate": 0.01,
  "discount_factor": 0.9,
  "exploration_factor": 0.2,
  "num_episodes": 29,
  "max_steps": 1,
  "verbose": 0,
  "notify_percent": 50
})

result = agent.train(env)

Started Training...

	Episode 0/29 0%
	Episode 29/29 100%

Done training!
